In [ ]:
import boto3

def compare_faces(sourceFile, targetFile):

    client = boto3.client('rekognition')

    imageSource = open(sourceFile, 'rb')
    imageTarget = open(targetFile, 'rb')

    response = client.compare_faces(SimilarityThreshold=0,
                                    SourceImage={'Bytes': imageSource.read()},
                                    TargetImage={'Bytes': imageTarget.read()})
    result = ""
    for faceMatch in response['FaceMatches']:
        position = faceMatch['Face']['BoundingBox']

        # 사용자를 위한 UI
        result = f"동일 인물일 확률 : {faceMatch['Similarity'] : .2f}%"

        # similarity = str(faceMatch['Similarity'])
        # print('The face at ' +
        #       str(position['Left']) + ' ' +
        #       str(position['Top']) +
        #       ' matches with ' + similarity + '% confidence')

    imageSource.close()
    imageTarget.close()
    if len(response['FaceMatches']) == 0:
        return "얼굴을 인식할 수 없습니다."
    else:
        return result
  
    return len(response['FaceMatches'])

def main():
    source_file = 'imgs/KakaoTalk_20250916_153811164.jpg'
    target_file = 'imgs/다운로드.jpeg'
    face_matches = compare_faces(source_file, target_file)
    print(face_matches)
    # print("Face matches: " + str(face_matches))

if __name__ == "__main__":
    main()

In [ ]:
# 모듈화 시켜서 배포
# aws.py
# └ class Rekognition
#   └compare_faces
#    └detect_faces
#     └detect_Label
#      └ recognize_celeb

In [ ]:
import boto3
import json

def detect_faces(photo):
    
    client = boto3.client('rekognition')

    with open(photo, "rb") as image:
        response = client.detect_faces(Image={'Bytes': image.read()},
                                   Attributes=['ALL'])
    result = ""
    # print('Detected faces for ' + photo)
    for faceDetail in response['FaceDetails']:
        # print('The detected face is between ' + str(faceDetail['AgeRange']['Low'])
        #       + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')

        # print('Here are the other attributes:')
        # print(json.dumps(faceDetail, indent=4, sort_keys=True))

        # Access predictions for individual face details and print them
       
        # 사용자를 위한 UI
        gender = faceDetail['Gender']['Value']
        # 성별 : 
        result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
        # 나이 :
        age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
        result += f"나이 : {age}(세)\n"
        # 감정 : 
        emotion = faceDetail['Emotions'][0]['Type']
        result += f"감정 : {emotion}"
        
        # print("Gender: " + str(faceDetail['Gender']))
        # print("Smile: " + str(faceDetail['Smile']))
        # print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
        # print("Face Occluded: " + str(faceDetail['FaceOccluded']))
        # print("Emotions: " + str(faceDetail['Emotions'][0]))

    if len(response['FaceDetails']) == 0:
        return "얼굴을 인식 할 수 없습니다"
    else:
        return result
    
    # return len(response['FaceDetails'])
    
def main():
    photo='imgs/KakaoTalk_20250916_153811164.jpg'
    face_count=detect_faces(photo)
    # print("Faces detected: " + str(face_count))
    print(face_count)

if __name__ == "__main__":
    main()

In [ ]:
import boto3
import random


def recognize_celebrities(photo):
    
    
    client = boto3.client('rekognition')

    with open(photo, 'rb') as image:
        response = client.recognize_celebrities(Image={'Bytes': image.read()})

    # print('Detected faces for ' + photo)
    result = ""
    # 닮은 연예인이 없는 경우.. 누가 들어도 괜찮은 사람을 닮았다고 리턴
    celeb_list = ["송중기", "아리아나 그란데", "아이유", "장원영", "카리나", "설윤", "송강", "강동원"]
    
    for celebrity in response['CelebrityFaces']:
        result = celebrity['Name']

    if len(response['CelebrityFaces']) == 0:
        # celeb_list[random.randrage(0, len(celeb_list) - 1)]
        return random.choice(celeb_list)
    else:
        return result
        
        #
    
        # print('Name: ' + celebrity['Name'])
        # print('Id: ' + celebrity['Id'])
        # print('KnownGender: ' + celebrity['KnownGender']['Type'])
        # print('Smile: ' + str(celebrity['Face']['Smile']['Value']))
        # print('Position:')
        # print('   Left: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Height']))
        # print('   Top: ' + '{:.2f}'.format(celebrity['Face']['BoundingBox']['Top']))
        # print('Info')
        # for url in celebrity['Urls']:
        #     print('   ' + url)
        # print()
    # return len(response['CelebrityFaces'])

def main():
    photo = 'imgs/KakaoTalk_20250916_153811164.jpg'
    celeb_count = recognize_celebrities(photo)
    print("Celebrities detected: " + str(celeb_count))

if __name__ == "__main__":
    main()

In [ ]:
import boto3

# 1) 이미지를 던졌을 때, 해당 label을 return
# 2) 이 이미지가 강아지인지 아닌지 확인하고 싶어서
#    -> "강아지일 확률은 95% 입니다.""
#    -> 혹은 "강아지가 아닙니다"

def detect_labels_local_file(photo):


    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
      
    result = ""
    
    for label in response['Labels']:
        result += f"{label['Name']} : {label['Confidence']:.2f}%\n"


    if len(response['Labels']) == 0:
        return "잘못된 이미지입니다."
    else:
        return result
    
def main():
    photo='imgs/랫서.jpeg'

    label_count=detect_labels_local_file(photo)
    print(label_count)


if __name__ == "__main__":
    main()





In [ ]:
import boto3

# 2) 이 이미지가 강아지인지 아닌지 확인하고 싶어서
#    -> "강아지일 확률은 95% 입니다.""
#    -> 혹은 "강아지가 아닙니다"

def detect_is_label(photo, key):


    client=boto3.client('rekognition')
   
    with open(photo, 'rb') as image:
        response = client.detect_labels(Image={'Bytes': image.read()})
        
      
    result = ""
    # 강아지는 Dog이라는 Key를 가지고 있다
    # 해당 Key가 존재한다면
    # 사용자에게 "강아지일 확률은 95.05%입니다" return
    # 해당 Key가 존재하지 않는다면
    # 사용자에게 "강아지가 아닙니다" return

    is_checked = False
    
    for label in response['Labels']:
        if label['Name'] == key:
            result += f"{key}일 확률은 {label['Confidence']:.2f}%\n"
            is_checked = True
            break

    if len(response['Labels']) == 0:
        return "잘못된 이미지입니다."

    elif is_checked == True:
        return result
        
    else:
        return f"{key}(이)가 아닙니다"
    
def main():
    photo='imgs/랫서.jpeg'

    key = "Dog"
    label_count=detect_is_label(photo, key)
    print(label_count)


if __name__ == "__main__":
    main()





In [ ]:
class Rekognition:
    def compare_faces(self, sourceFile, targetFile):

        client = boto3.client('rekognition')
    
        imageSource = open(sourceFile, 'rb')
        imageTarget = open(targetFile, 'rb')
    
        response = client.compare_faces(SimilarityThreshold=0,
                                        SourceImage={'Bytes': imageSource.read()},
                                        TargetImage={'Bytes': imageTarget.read()})
        result = ""
        for faceMatch in response['FaceMatches']:
            position = faceMatch['Face']['BoundingBox']
    
            # 사용자를 위한 UI
            result = f"동일 인물일 확률 : {faceMatch['Similarity'] : .2f}%"
    
            # similarity = str(faceMatch['Similarity'])
            # print('The face at ' +
            #       str(position['Left']) + ' ' +
            #       str(position['Top']) +
            #       ' matches with ' + similarity + '% confidence')
    
        imageSource.close()
        imageTarget.close()
        if len(response['FaceMatches']) == 0:
            return "얼굴을 인식할 수 없습니다."
        else:
            return result
      
        return len(response['FaceMatches'])


    def detect_faces(self, photo):
        
        client = boto3.client('rekognition')
    
        with open(photo, "rb") as image:
            response = client.detect_faces(Image={'Bytes': image.read()},
                                       Attributes=['ALL'])
        result = ""
        # print('Detected faces for ' + photo)
        for faceDetail in response['FaceDetails']:
            # print('The detected face is between ' + str(faceDetail['AgeRange']['Low'])
            #       + ' and ' + str(faceDetail['AgeRange']['High']) + ' years old')
    
            # print('Here are the other attributes:')
            # print(json.dumps(faceDetail, indent=4, sort_keys=True))
    
            # Access predictions for individual face details and print them
           
            # 사용자를 위한 UI
            gender = faceDetail['Gender']['Value']
            # 성별 : 
            result += f"성별 : {'남성' if gender == 'Male' else '여성'}\n"
            # 나이 :
            age = (faceDetail['AgeRange']['Low'] + faceDetail['AgeRange']['High']) // 2
            result += f"나이 : {age}(세)\n"
            # 감정 : 
            emotion = faceDetail['Emotions'][0]['Type']
            result += f"감정 : {emotion}"
            
            # print("Gender: " + str(faceDetail['Gender']))
            # print("Smile: " + str(faceDetail['Smile']))
            # print("Eyeglasses: " + str(faceDetail['Eyeglasses']))
            # print("Face Occluded: " + str(faceDetail['FaceOccluded']))
            # print("Emotions: " + str(faceDetail['Emotions'][0]))
    
        if len(response['FaceDetails']) == 0:
            return "얼굴을 인식 할 수 없습니다"
        else:
            return result
    
    def recognize_celebrities(self, photo):
        
        
        client = boto3.client('rekognition')
    
        with open(photo, 'rb') as image:
            response = client.recognize_celebrities(Image={'Bytes': image.read()})
    
        # print('Detected faces for ' + photo)
        result = ""
        # 닮은 연예인이 없는 경우.. 누가 들어도 괜찮은 사람을 닮았다고 리턴
        celeb_list = ["송중기", "아리아나 그란데", "아이유", "장원영", "카리나", "설윤", "송강", "강동원"]
        
        for celebrity in response['CelebrityFaces']:
            result = celebrity['Name']
    
        if len(response['CelebrityFaces']) == 0:
            # celeb_list[random.randrage(0, len(celeb_list) - 1)]
            return random.choice(celeb_list)
        else:
            return result
    
    def detect_labels_local_file(self, photo):
    
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
          
        result = ""
        
        for label in response['Labels']:
            result += f"{label['Name']} : {label['Confidence']:.2f}%\n"
    
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다."
        else:
            return result
    
    def detect_is_label(self, photo, key):
    
    
        client=boto3.client('rekognition')
       
        with open(photo, 'rb') as image:
            response = client.detect_labels(Image={'Bytes': image.read()})
            
          
        result = ""
        # 강아지는 Dog이라는 Key를 가지고 있다
        # 해당 Key가 존재한다면
        # 사용자에게 "강아지일 확률은 95.05%입니다" return
        # 해당 Key가 존재하지 않는다면
        # 사용자에게 "강아지가 아닙니다" return
    
        is_checked = False
        
        for label in response['Labels']:
            if label['Name'] == key:
                result += f"{key}일 확률은 {label['Confidence']:.2f}%\n"
                is_checked = True
                break
    
        if len(response['Labels']) == 0:
            return "잘못된 이미지입니다."
    
        elif is_checked == True:
            return result
            
        else:
            return f"{key}(이)가 아닙니다"

In [ ]:
from aws import detect_is_label
result = detect_is_label("imgs/냥이.jpeg", "Cat")
print(result)

In [ ]:
# API KEY
# 특정 서버에 GET 방식으로 요청(request) : Client
# Server가 응답(response)

flatbuffers               24.3.25
tensorflow_cpu            2.18.1


In [5]:
!pip install flask

  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)

   ---------------------------------------- 3/3 [flask]



In [ ]:
# 웹 프레임워크
# 1) Flask : 마이크로 웹 프레임워크
# 2) Django
import flask
from flask import Flask # 서버를 라우팅, 구동
app = Flask(__name__)
@app.route("/")
def index():
    return "<h1>성균의 웹 페이지</h1>"
if __name__ == "__main__":

    # 127.0.0.1은 IPv4 이다
    # IP란? Server가 어디에 있는지 알려주는 주소
    # 네이버 서버 -> 25.50.127.255라고 하면,
    # 영문명(도메인) -> IPv4 변환
    # 127.0.0.1 (Localhost) -> "나"를 가리킴
    # 누군가가 나를 바라보게 하고 싶어요
    # 공개(Public) IP가 필요하다
    # 공개 IPv4는 2^32 -> 42~3억개만 전 세계에 존재
    
    # 개인(Private) IPv4를 통해 다른 사람 서버를 바라보자
    # 같은 인터넷망(공유기)에서는 개인 IPv4를 바라볼 수 있음
    
    # 0.0.0.0 : 누구나 접속 가능
    app.run(host="0.0.0.0")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.10.141.13:5000
Press CTRL+C to quit
10.10.141.249 - - [17/Sep/2025 10:58:24] "GET / HTTP/1.1" 200 -


In [ ]:
# 이 사진이 고양이 사진인지 판별
from aws import Rekognition
result = Rekognition().detect_is_label("imgs/냥이.jpeg", "Cat")
print(result)